Modeling attempt for one step ethanol to olefins following Wang et al., Green Chem., 2021. 23, 9534 (10.1039/d1gc02854e)

In [1]:
import qsdsan as qs, biosteam as bst, thermosteam as tmo, numpy as np, pandas as pd
from qsdsan.sanunits import _heat_exchanging
bst.nbtutorial()



ImportError: cannot import name 'TicToc' from 'biosteam.utils' (c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\utils\__init__.py)

In [2]:
import biosteam, qsdsan, thermosteam
print("biosteam:", biosteam.__version__)
print("qsdsan  :", qsdsan.__version__)
print("thermo  :", thermosteam.__version__)
import sys; print(sys.executable)
import biosteam.utils as u; print("has TicToc?", hasattr(u, "TicToc"))

ImportError: cannot import name 'TicToc' from 'biosteam.utils' (c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\utils\__init__.py)

In [33]:
#from atj_saf import atj_baseline
#import atj_chemicals

from atj_saf.atj_baseline.atj_chemicals import *
from atj_saf.atj_one_step.equipments import *
from atj_saf.atj_one_step.units import *

In [26]:
bst.settings.CEPCI = 800.8

In [3]:
qs.set_thermo(chemicals)

In [4]:
# Parameters from Wang et al 2021 Table S3, except noted otherwise
etoh_conc = 1           # pure ethanol
etoh_price  = 1.79      # $/gal from Tasble S4 [Corn EtOH]
olig_cat_whsv = 1       # /h
eto_cat_whsv = 1        # /h 
olig_cat_replace = 1    # /yr
olig_cat_cost = 154     # $/kg
eto_cat_replace = 1     # /yr
eto_cat_cost = 60       # $/kg
eto_temp = 350          # C
etoh_in_T = 293         # C from Humbird et al 2011 pg 128
etoh_in_P = 101325      # bar from Humbird et al 2011 pg 128
etoh_production = 39    # MM gal/yr

In [5]:
# ETO conditions from pg 9541 of Wang et al 2022 for basecase except noted otherwise
eto_temp = 623      # K
eto_conv = 1       # 100%
eto_yield  = 0.58   # 0.58 GGE per gal etoh 
eto_c3plus_sel = 0.88   # 88% selectivity to C3+ olefins  [Table 1 Zhang et al ACS Catalysis]
eto_ethylene_sel = 0.91 # 9% ethylene selectivity [Table 1 Zhang et al ACS Catalysis]
eto_propene_sel = 0.05 # 5% ethylene selectivity [Table 1 Zhang et al ACS Catalysis]
eto_butene_sel = 0.66 # 66% butene selectivity [Table 1 Zhang et al ACS Catalysis]
eto_aa_sel = 0.019 # 1.9% acetaldehyde selectivity [Table 1 Zhang et al ACS Catalysis]
eto_c5plus_sel = 0.17 # 17% C5+ selectivity [Table 1 Zhang et al ACS Catalysis]

In [6]:
chemicals['Ethanol'].rho(T = 300, P = 101325, phase = 'l')

783.5691831230228

In [7]:
from units import u, Q_

# Canonical parameter
ethanol_prod = Q_(39, "mgpy")              # 39 million gal/yr

UndefinedUnitError: 'mgpy' is not defined in the unit registry

In [ ]:
ethanol_prod = Q_(39e6, "gallon/year")  # 39 million gal/yr

print(ethanol_prod.to("liter/year"))
print(ethanol_prod.to("meter**3/second"))

147631059.57599998 liter / year
0.004678145979922427 meter ** 3 / second


In [ ]:
distance

<Quantity(24.8, 'meter')>

In [ ]:
'mgpy' in u

True

In [ ]:
from units import u, Q_

ethanol_prod = Q_(39, "mgpy")  # 39 million gal per year

ethanol_gpy  = ethanol_prod.to("gallon/year")
ethanol_Ly   = ethanol_prod.to("liter/year")
ethanol_m3_s = ethanol_prod.to("meter**3/second")

print(ethanol_gpy)
print(ethanol_Ly)
print(ethanol_m3_s)

UndefinedUnitError: 'million' is not defined in the unit registry

In [ ]:
ethanol_prod

<Quantity(39, 'mgpy')>

In [ ]:
etoh_production

39

In [8]:


# Inlet ethanol
etoh_in = qs.SanStream('Ethanol_In', 
                       Ethanol = etoh_production*1e6*(1/264.17)*789*(1/330/24)*etoh_conc,
                       Water = etoh_production*1e6*(1/264.17)*789*(1/330/24)*(1-etoh_conc),
                       phase = 'l',
                       T = etoh_in_T,
                       P = etoh_in_P,
                       units = 'kg/hr',
                       price = etoh_price*264.17*(1/789)) # USD/gal

In [9]:
furnace_1 = _heat_exchanging.HXutility('Furnace_1',
                                       ins = etoh_in,
                                       T = eto_temp,
                                       rigorous = True

)
furnace_1.simulate()

In [ ]:
eto = OligomerizationReactor(ins = furnace_1.outs[0],
                             conversion = eto_conv,
                             temperature = eto_temp,
                             pressure = 101325,
                             WHSV = eto_cat_whsv,
                             catalyst_price = olig_cat_cost)

In [ ]:
eto.simulate()

In [ ]:
# 1) Gas phase dehydration of ethanol to ethylene 
dehydration_rxn = bst.Reaction('Ethanol,g -> Water,g + Ethylene,g', reactant = 'Ethanol', 
                            X = 1, phases = 'lg',  basis = 'mol')

In [ ]:
dehyd_reactor = IsothermalReactor(ins = furnace_1.outs[0],
                            conversion = eto_conv,
                             temperature = eto_temp,
                             pressure = 101325,
                             WHSV = eto_cat_whsv,
                             catalyst_price = olig_cat_cost, 
                             reaction = dehydration_rxn)
dehyd_reactor.simulate()
                          

In [ ]:
dehyd_reactor.results()

Isothermal reactor                                Units       R3
Natural gas         Duty                          kJ/hr  1.5e+07
                    Flow                        kmol/hr     28.9
                    Cost                         USD/hr      101
Design              Catalyst Weight                  kg 1.47e+04
                    Volume                            L 4.23e+04
                    Vessel Weight                    lb 1.46e+04
                    Length                           ft     25.8
                    Diameter                         ft     8.59
                    Wall thickness                   in    0.438
                    Duty                          kJ/hr  1.5e+07
Purchase cost       Horizontal pressure vessel      USD 8.27e+04
                    Platform and ladders            USD 3.52e+03
                    Catalyst                        USD 2.26e+06
Total purchase cost                                 USD 2.35e+06
Utility cost                                     USD/hr      101
Additional OPEX                                  USD/hr      287

In [ ]:
eto.outs[0].show(flow = 'kg/hr')

SanStream: ss1 from <OligomerizationReactor: R1>
phase: 'l', T: 623 K, P: 101325 Pa
flow (kg/hr): Ethylene      1.34e+03
              Propene       735
              Butene        9.85e+03
              Pentene       2.5e+03
              Acetaldehyde  279


In [ ]:
eto.results()

Oligomerization reactor                           Units       R1
Natural gas         Duty                          kJ/hr 8.53e+07
                    Flow                        kmol/hr      165
                    Cost                         USD/hr      575
Design              Catalyst Weight                  kg 1.47e+04
                    Volume                            L 4.23e+04
                    Vessel Weight                    lb 1.46e+04
                    Length                           ft     25.8
                    Diameter                         ft     8.59
                    Wall thickness                   in    0.438
                    Duty                          kJ/hr 8.53e+07
Purchase cost       Horizontal pressure vessel      USD 7.87e+04
                    Platform and ladders            USD 3.52e+03
                    Catalyst                        USD 2.26e+06
Total purchase cost                                 USD 2.35e+06
Utility cost                                     USD/hr      575
Additional OPEX                                  USD/hr      287

In [10]:
# One step ETO
eto_rxn = bst.ParallelReaction([
# Reaction definition                                     # Reactant                    # Conversion
bst.Reaction('2Ethanol,g + Hydrogen,g -> Butene,g + 2H2O,g' ,            reactant = 'Ethanol',     X = 0.66,    basis = 'wt',  phases = 'lg',  correct_atomic_balance = True),
bst.Reaction('1.5Ethanol,g + 0.75Hydrogen,g -> Propene,g + 1.5H2O,g',    reactant = 'Ethanol',     X = 0.05,  basis = 'wt',  phases = 'lg',  correct_atomic_balance = True),
bst.Reaction('2.5Ethanol,g + 1.25Hydrogen,g -> Pentene,g + 2.5H2O,g',    reactant = 'Ethanol',     X = 0.17,  basis = 'wt',  phases = 'lg',  correct_atomic_balance = True),
bst.Reaction('Ethanol,g -> Ethylene,g + H2O,g',                          reactant = 'Ethanol',     X = 0.091,  basis = 'wt',  phases = 'lg',  correct_atomic_balance = True),
bst.Reaction('Ethanol,g -> Acetaldehyde,g + Hydrogen,g',                 reactant = 'Ethanol',     X = 0.019,  basis = 'wt',  phases = 'lg',  correct_atomic_balance = True)
])



In [ ]:
eto_rxn

ParallelReaction (by wt):
index  stoichiometry                                                      reactant     X[%]
[0]    Ethanol,g -> 0.391 Water,g + 0.609 Butene,g + 1.4e-17 Hydrogen,g   Ethanol,g   66.00
[1]    Ethanol,g -> 0.391 Water,g + 0.609 Propene,g + 1.4e-17 Hydrogen,g  Ethanol,g    5.00
[2]    Ethanol,g -> 0.391 Water,g + 0.609 Pentene,g + 1.4e-17 Hydrogen,g  Ethanol,g   17.00
[3]    Ethanol,g -> 0.391 Water,g + 0.609 Ethylene,g                      Ethanol,g    9.10
[4]    Ethanol,g -> 0.0438 Hydrogen,g + 0.956 Acetaldehyde,g              Ethanol,g    1.90


In [27]:
eto_2 = IsothermalReactor(ins = furnace_1.outs[0],
                            conversion = eto_conv,
                             temperature = eto_temp,
                             pressure = 101325,
                             WHSV = eto_cat_whsv,
                             catalyst_price = olig_cat_cost, 
                             reaction = eto_rxn)
                          

In [28]:
eto_2.simulate()

In [29]:
eto_2.results()

Isothermal reactor                                Units       R4
Natural gas         Duty                          kJ/hr 1.07e+05
                    Flow                        kmol/hr    0.206
                    Cost                         USD/hr    0.721
Design              Catalyst Weight                  kg 1.47e+04
                    Volume                            L 4.23e+04
                    Vessel Weight                    lb 1.46e+04
                    Length                           ft     25.8
                    Diameter                         ft     8.59
                    Wall thickness                   in    0.438
                    Duty                          kJ/hr 1.07e+05
Purchase cost       Horizontal pressure vessel      USD 8.27e+04
                    Platform and ladders            USD 3.52e+03
                    Catalyst                        USD 2.26e+06
Total purchase cost                                 USD 2.35e+06
Utility cost                                     USD/hr    0.721
Additional OPEX                                  USD/hr      287

In [30]:
eto_2

IsothermalReactor: R4
ins...
[0] s1  from  HXutility-Furnace_1
phases: ('g', 'l'), T: 623 K, P: 101325 Pa
flow (g/hr): (g) Ethanol  1.47e+07
outs...
[0] s12
phases: ('g', 'l'), T: 623 K, P: 101325 Pa
flow (g/hr): (g) Water         5.58e+06
                    Ethanol       1.47e+05
                    Ethylene      8.15e+05
                    Propene       4.48e+05
                    Butene        5.91e+06
                    Pentene       1.52e+06
                    Hydrogen      1.22e+04
                    Acetaldehyde  2.67e+05


In [21]:
eto_4 = TrialReactor(ins = furnace_1.outs[0],
                            conversion = eto_conv,
                             temperature = eto_temp,
                             pressure = 101325,
                             WHSV = eto_cat_whsv,
                             catalyst_price = olig_cat_cost, 
                             orientation = 'Vertical', material = 'ss',
                             reaction = eto_rxn)
                          

In [31]:
(ins = furnace_1.outs[0],
                            conversion = eto_conv,
                             temperature = eto_temp,
                             pressure = 101325,
                             WHSV = eto_cat_whsv,
                             catalyst_price = olig_cat_cost, 
                             reaction = eto_rxn)

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (2926635667.py, line 1)

In [22]:
eto_4.simulate()

In [23]:
eto_4.results()

Trial reactor                                   Units       R3
Natural gas         Duty                        kJ/hr 1.07e+05
                    Flow                      kmol/hr    0.206
                    Cost                       USD/hr    0.721
Design              Catalyst Weight                kg 1.47e+04
                    Volume                          L 4.23e+04
                    Vessel Weight                  lb 1.46e+04
                    Length                         ft     25.8
                    Diameter                       ft     8.59
                    Wall thickness                 in    0.438
                    Duty                        kJ/hr 1.07e+05
                    Pressure                      psi     14.7
Purchase cost       Vertical pressure vessel      USD 6.01e+04
                    Platform and ladders          USD    2e+04
                    Catalyst                      USD 2.26e+06
Total purchase cost                               USD 2.35e+06
Utility cost                                   USD/hr    0.721
Additional OPEX                                USD/hr      287

In [24]:
eto_4

TrialReactor: R3
ins...
[0] s1  from  HXutility-Furnace_1
phases: ('g', 'l'), T: 623 K, P: 101325 Pa
flow (g/hr): (g) Ethanol  1.47e+07
outs...
[0] s9
phases: ('g', 'l'), T: 623 K, P: 101325 Pa
flow (g/hr): (g) Water         5.58e+06
                    Ethanol       1.47e+05
                    Ethylene      8.15e+05
                    Propene       4.48e+05
                    Butene        5.91e+06
                    Pentene       1.52e+06
                    Hydrogen      1.22e+04
                    Acetaldehyde  2.67e+05
